# Milestone 2

## Introduction

We are going to analyze the dataset, cleanliness, blabla, start to explore the question.

ntm lucas

## Dataset overview

Size of dataset, blablabla

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import statsmodels.formula.api as sm

In [ ]:
OPEN_FOOD_FACTS_PATH = 'data/en.openfoodfacts.org.products.csv'
food_facts_df = pd.read_csv(OPEN_FOOD_FACTS_PATH, delimiter='\t',low_memory=False)
food_facts_df.head(5)

### Available fields

The dataset description is available [here](https://static.openfoodfacts.org/data/data-fields.txt).



This dataset is provided with a text file describing the different fields as follows :

* fields that end with _t are dates in the UNIX timestamp format (number of seconds since Jan 1st 1970)
* fields that end with _datetime are dates in the iso8601 format: yyyy-mm-ddThh:mn:ssZ
* fields that end with _tags are comma separated list of tags (e.g. categories_tags is the set of normalized tags computer from the categories field)
* fields that end with a language 2 letter code (e.g. fr for French) is the set of tags in that language
* fields that end with _100g correspond to the amount of a nutriment (in g, or kJ for energy) for 100 g or 100 ml of product
* fields that end with _serving correspond to the amount of a nutriment (in g, or kJ for energy) for 1 serving of the product

**TODO**: show data

### General cleaning 


Each product of the dataset has a field named `code`. This should be unique to each row:

In [ ]:
print("We have {} rows in the dataset".format(food_facts_df.shape[0]))
print("We have {} different code in the dataset".format((np.unique(food_facts_df['code']).shape[0])))

Indeed, the code is unique for each row of the dataset except for a small number of duplicates. Let's check that. 

In [ ]:
number_duplicates=food_facts_df['code'].duplicated()
print("We have {} duplicated code".format(number_duplicates.value_counts()[1]))

Let's start by dropping these duplicated rows:

In [ ]:
food_facts_df.drop_duplicates(subset=['code'],inplace=True)
print("We have {} rows in the dataset after dropping the duplicates.".format(food_facts_df.shape[0]))

Great! We have now a unique code for each row. We can set the code as an index for the whole dataset.

In [ ]:
#lets fix the code as indes 
food_facts_df.set_index('code', inplace=True)

In [ ]:
food_facts_df.head(5)

As we can see from the 5 first rows in the dataset, the data is messy and has a huge number of NaNs. We need a huge data cleaning.

### Data completeness

Sparse dataset, lots of quasi empty columns (Lucas)

### Reliability of the dataset



The creators of each row of the dataset are provided as a field. Let's try to see who are the main contributors to this data before starting to analyze it.

In [ ]:
## Computing the number of contributors
nb_contributors=food_facts_df['creator'].unique().shape[0]
print("We have overall {} contributors".format(nb_contributors))

Let's see how many contributions each creator has in the dataset

In [ ]:
# Computing the contributions 
counts = food_facts_df['creator'].value_counts()
counts = counts.to_frame(name='counts')
counts['percentage'] = counts['counts']/food_facts_df.shape[0]

# Extracting only the top find and calculating the percentage of contribution
top_10 = round(sum(counts[0:10]['percentage']),2)*100
print("The 10 main", 'creators',"account for", top_10,"% of all contributions \n")
print(counts.head(10))

After googling these 10 main contributors we can easily identify them: 
* `kiliweb` represents the YUKA app (a French app aiming to compare the products based on different criterions).
* `usda` represnets United States Department of Agriculture. 
* `openfoodfacts-contributors` a bench of 20 main contributors.
* `data-limite-app` food application available for ios and androids users (French app).
* `Elcoco` also a food application (spanish app).
* `openfood-ch-import` seems to be openfood contributors from Switzerland.
* `sebleouf`, `tacite` , `tacinte` and `waistline-app` are unknown contributors. 

The first 6 sources of contributions seem to be reliable since we can clearly understand where the data comes from.

In [ ]:
#Getting the top 6
top5_cont=counts.head(6).index

Since the first 6 editors have 85% of the total contributions and are reliable. We will base our analysis on their contributions.

In [ ]:
food_facts_df=food_facts_df.loc[food_facts_df['creator'].isin([top5_cont[0],top5_cont[1],top5_cont[2],top5_cont[3],top5_cont[4],top5_cont[5]])]

In [ ]:
print('There are {:,} rows '.format(food_facts_df.shape[0]) + 'and {} columns in our data'.format(food_facts_df.shape[1]))

## Question 1: How do nutrients influence the selected nutrition scores?

The majority of the dataser fields are nutrients. We will try in this part to see their impact on the nutrition score. This analysis requires an additional cleaning of the dataset. 

### Additional cleaning

First, we will see the data type of each field: 

In [ ]:
food_facts_df.dtypes

We will discard all no-float fields since they cannot represent the nutrients. (ie:object)

In [ ]:
food_facts_df = food_facts_df.select_dtypes(include=[np.float])

In [ ]:
food_facts_df.columns

In [ ]:
print('There are {:,} rows '.format(food_facts_df.shape[0]) + 'and {} columns left in our data'.format(food_facts_df.shape[1]))

Let us now analyze the level of completness for each editor

In [ ]:
food_facts_df.head()

As we can see, our dataset has a huge number of NaNs. Dropping all of them will lead to an empty dataframe. 

#### Dealing with missing values
As an efficient strategy to deal with these missing values, we will first count the number of NaNs from the dataset and then remove the features where the number of missing values exceeds a certain threshold. 

Let's count the number of nans for each feature: 

In [ ]:
nutrition_facts_df=food_facts_df.copy()

In [ ]:
NaNs_distribution_series=nutrition_facts_df.isnull().sum().sort_values()
NaNs_distribution_series

Let us visualize the distribution of NaNs in the dataset

In [ ]:
nutrition_facts_df.isnull().sum().plot(kind='hist', figsize=(12,5),bins=50)
plt.title('Distribution of NaNs')
plt.xlabel('NaNs')

plt.show()

Many columns have a huge number of missing values(More than 85 columns with 350000 NaNs), let's try to remove some of them by playing with the threshold value.   
We will plot the new distribution of NaNs. Our main objective is to remove this huge peak of frequency visualized in the plot above.   
We tune the threshold with a trial and error to have a satisfactory result. 

In [ ]:
nutrition_facts_thresh_df = nutrition_facts_df.dropna(axis=1, thresh= len(nutrition_facts_df)*0.2,how='all')
nutrition_facts_thresh_df = nutrition_facts_thresh_df.dropna(axis=0, how='all')

print('There are now {:,} rows '.format(nutrition_facts_thresh_df.shape[0]) + 'and {} columns left in our data'.format(nutrition_facts_thresh_df.shape[1]))

In [ ]:
nutrition_facts_thresh_df.isnull().sum().plot(kind='hist', figsize=(10,5),bins=50)
plt.title('Distribution of NaNs')
plt.xlabel('NaNs')

plt.show()

We will se the total number of missing values for each field in the dataset after filtering them. 

In [ ]:
nutrition_facts_thresh_df.isnull().sum().sort_values()

In [ ]:
print ("We have now {} of samples with {} fields".format(nutrition_facts_thresh_df.shape[0],
                                                         nutrition_facts_thresh_df.shape[1]))

We think that this is the optimal way to decrease the number of NaNs in this dataset. Putting the threshold higher then the selected value will delete the nutrition score columns. 

After dealing with the missing values, let's check the statistics of our data. 

#### Looking at the stats!

In [ ]:
nutrition_facts_thresh_df.describe()

Well, It seems that there is no difference between the `nutrition-score-fr_100g` and `nutrition-score-uk_100g`. Let's analyse that. 

In [ ]:
nutrition_facts_thresh_df[['nutrition-score-fr_100g','nutrition-score-uk_100g']].select_dtypes(include=float).plot(kind='box', subplots=True, title='Nutrition score uk and fr boxplots', figsize=(8,5))

plt.show()

As expected, the two scores has the same distribution except some outliers in the `nutrition-score-fr_100g`.

#### Dealing with outliers

Visualizing the stats. We see a huge number of outliers in the dataset after looking to min, max, mean and standard deviation of each feature.
All the ingredient features should have a reasonable value <=100g except the energy which is expressed in kJ. 

First, the column `serving_quantity` is not an ingredient. It can be dropped 

In [ ]:
nutrition_facts_thresh_df=nutrition_facts_thresh_df.drop(columns=['serving_quantity'])

In [ ]:
# storing the columns fields in a list
fields=list(nutrition_facts_thresh_df.columns)

Let us now remove the outliers. All nutrition feautres should have a value <=100g (except energy_100g)

In [ ]:
nutrition_facts_clean_df= nutrition_facts_thresh_df[(nutrition_facts_thresh_df
                                                      [nutrition_facts_thresh_df.columns.difference(['energy_100g'])] <= 100.0).all(1)]


Negative values can not be possible. Except for the nutrition score that is by definition defined between -14 and 40 

In [ ]:

nutrition_facts_clean_df = nutrition_facts_clean_df[(nutrition_facts_clean_df[nutrition_facts_clean_df.columns.difference(['nutrition_score_fr_100g', 'nutrition_score_uk_100g'])] >= 0).all(1)]

In [ ]:
print("We have only {} rows left".format(nutrition_facts_clean_df.shape[0]))

We lost a huge number of rows by removing the outliers. The dataset is highly corrupted in terms of ingredients.  We are going to see later if this number of samples is enough to have significant results. 

In [ ]:
nutrition_facts_clean_df.head(10)

We lost a huge number of rows but the dataset is now very clean !! 

### Data exploration

Now we have a clean dataset. Our strategy to find the impact of each variable on the nutrition score is the following: 

* 1) Correlation analysis between the variables 
* 2) Performing ordinary least square (OLS) regression using statsmodel 
* 3) Coefficient analysis


Before analyzing the impact of our variables on the nutrition score, we need to normalize our data by removing the mean and dividng by the standard deviation 

In [ ]:
nutrition_facts_clean_df = nutrition_facts_clean_df.select_dtypes(include=float).transform(lambda x: (x - x.mean()) / x.std())

We can check now if the mean of our variables is close to zero and the standard deviation is close to 1. 

In [ ]:
nutrition_facts_clean_df.describe()

Great! our data is now ready for the analysis on the impact of our variables on the nutrition score.

Before doing so, let's see the correlation between variables visualized in a heatmap.

#### Correlation analysis

In [ ]:
correlation_variables = nutrition_facts_clean_df.corr(method = "pearson")

f, ax = plt.subplots(figsize=(10, 10))

sns.heatmap(correlation_variables, mask=np.zeros_like(correlation_variables, dtype=np.bool),cmap=sns.diverging_palette(200, 10, as_cmap=True), square=True, ax=ax)

From the plot of correlation above we can confirm first that `nutrition-score-fr_100g` and `nutrition-score-uk_100g` are nearly the same.    
We can see also a  correlation equal to 1 between `sodium_100g` and `salt_100g`. This is why we will drop the column `sodium_100g` to avoid redundancy. 

In [ ]:
#We drop sodium column. 
nutrition_facts_clean_df=nutrition_facts_clean_df.drop(columns=['sodium_100g'])
fields_left=nutrition_facts_clean_df.columns

In [ ]:
#We drop one of the nutrition scores 
nutrition_facts_clean_df=nutrition_facts_clean_df.drop(columns=['nutrition-score-uk_100g'])


In [ ]:
fields_left=nutrition_facts_clean_df.columns

Let's see the heatmap again. 

In [ ]:
#calculating pearson correlation
correlation_variables = nutrition_facts_clean_df.corr(method = "pearson")

f, ax = plt.subplots(figsize=(10, 10))

#we draw the heatmap using seaborn
sns.heatmap(correlation_variables, mask=np.zeros_like(correlation_variables, dtype=np.bool),cmap=sns.diverging_palette(200, 10, as_cmap=True), square=True, ax=ax)

By looking at the heatmap, we can already see that `energey_100g`, `fat_100g`, `saturated_fat_100g` and `sugar_100g` have the highest correlation on the nutrition score.


Now everything is ready. What we want to do is to find the impact of each ingredient on the nutrition score. One way to do that is a regression. It allows us to see the impact of each variable on this score by looking to the weights. 

#### OLS Regression

Let's prepare our formula for the regression: 

We rename the columns by replacing '-' by '_'. This will allow us to create the formula for the statsmodel regression since'-' is interpreted as a math operation while definiing the formula as a string. 

In [ ]:
for field in fields: 
    nutrition_facts_clean_df = nutrition_facts_clean_df.rename(columns={field: field.replace('-','_')})
fields=nutrition_facts_clean_df.columns

In [ ]:
nutrition_facts_clean_df.head()

In [ ]:
fields_left

In [ ]:
# OLS regression formula 
fields_left=nutrition_facts_clean_df.columns
formula_string="nutrition_score_fr_100g ~ "
for field in fields_left[:-1]:
    formula_string=formula_string+field+"+"
formula_string=formula_string[:-1]
formula_string

In [ ]:
nutrition_score_regression = sm.ols(formula= formula_string, data = nutrition_facts_clean_df)
res = nutrition_score_regression.fit()
print(res.summary())

### Regression results 

Analysing the regression results from the coeficents of each ingredient, we can deduce the impact of these ingredients on the the nutrition score.  

#### Positive coef 
* `sugars_100g` has the highest positive impact nutrition score ~ 0.37  
* `saturated_fat_100g`has the fourth highest impact on nutrition socre ~ 0,36
* `energy_100` has the second highest nutrition score ~ 0.18 
* `fat_100g` with a coef ~ 0.16
* `salt_100g` with a coef ~ 0.16
* `nova_group`with a coef ~ 0.12


#### Negative coef 
* `fiber_100g` has the highest negative nutrition score ~-0.2 

#### Zero coef
* `ingredients_from_palm_oil_n ` , `additives_n  ` and `carbohydrates_100g` don't have any impact on the nutrition score 

From the french nutrition score datasheet page 24, we see that the highest the nutrition score is the lowest the nutritional quality is. We can then deduce that all products with a high percentage of energy, sugar, salt, and saturated fat all unhealthy and products with a high percentage of fiber are healthy. All other ingredients do not have a huge impact on the nutrition score.  
By analyzing the p-value we can deduce that our variables are statistically significant except the `ingredients_that_may_be_from_palm_oil_n`.    
The number of samples we have in hand after dropping the outliers and NaNs is enough to have significant results.

One of the features which seems to have a high impact on the nutrition score is the nova group. The values of this group are {1,2,3,4}

* Group 1 - Unprocessed or minimally processed foods :Unprocessed (or natural) foods are edible parts of plants (seeds, fruits, leaves, stems, roots) or of animals (muscle, offal, eggs, milk), and also fungi, algae and water, after separation from nature.
* Group 2 - Processed culinary ingredients : Processed culinary ingredients, such as oils, butter, sugar and salt, are substances derived from Group 1 foods or from nature by processes that include pressing, refining, grinding, milling and drying.
* Group 3 - Processed foods : Processed foods, such as bottled vegetables, canned fish, fruits in syrup, cheeses and freshly made breads, are made essentially by adding salt, oil, sugar or other substances from Group 2 to Group 1 foods. 
* Group 4 - Ultra-processed food and drink products: Ultra-processed foods, such as soft drinks, sweet or savoury packaged snacks, reconstituted meat products and pre-prepared frozen dishes.  

Beeing in a higher group leads also to a high nutrition score and then low nutritional quality.  

#### Conclusion

We can conclude that the analysis of the nutrients on the nutrition score is giving a satisfactory results. The main problem we encontered here is the missing values. We were forced to loose a huge number of rows and fetures from the dataset.  
We think that maybe some products don't contain some of the ingredients so the editor didn't  set the corresponding value to zero and thus added a missing value to the dataset. This can explain the huge number of missing values in the dataset. 

## Question 2: Comparing bio vs. normal products, is there a real difference in composition?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we keep the question, further investigations

## Question 3: What are the levels of interdependencies between nations based on their production and importations?

### Additional cleaning

### Data exploration

### Results

quick conclusion, we **don't keep** the question, further investigations

## Conclusion: proposed data story